# Extract Transform Load #

In [ ]:
# Python
import os

# Third part
import pandas as pd
pd.set_option('display.max_columns', 15)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:,.2f}'.format
import sqlite3

# Support etl library
from support import etl, datasetup, orders
from support.orders import Orders

In [ ]:
# Run ETL
context = {
    'data': 'data'
    , 'orders_filepath': 'orders'
    , 'orders_file': 'Backup_orders_after_etl.csv'
    , 'rates_file': 'rates.csv'
    , 'to_replace_company_names': datasetup.find_company
    , 'value_company_names': datasetup.replace_company
    , 'stores': datasetup.stores
    , 'channels': datasetup.channels
}
event = {
    're_generate_rates': False,
    're_generate_orders_csv': False
}
df = etl.e_t_l(context, event)

In [ ]:
# find one of the newly imported

order_id = '#727811-US'

my_orders = Orders(df)

display(my_orders.get_order_header(order_id).set_index('order_id').T)
print()
display(my_orders.get_order_lines(order_id))

In [ ]:
# dump one of the newly imported orders in json format

order_id = '#728311-US'

my_orders = Orders(df)

order_json = my_orders.get_order_json(order_id)

In [ ]:
# find one order from the database

conn = sqlite3.connect(os.path.join(context['data'],('orders_pipeline.db')))
with conn:
    c = conn.cursor()
    sql_stm = "SELECT * FROM orders"
    c.execute(sql_stm)
    orders_table = c.fetchall()
    sql_stm = 'PRAGMA table_info(orders)'
    c.execute(sql_stm)
    cols = [col[1] for col in c.fetchall()]
    df_sql = pd.DataFrame(orders_table, columns = cols)
    
#sample order with lines 711711, 692511, 681211, 237622, 119233, 2137, 722511

order_id = '#725311-US'

my_orders = Orders(df_sql)
display(my_orders.get_order_header(order_id).set_index('order_id').T)
print()
display(my_orders.get_order_lines(order_id))